# DiploDatos 2019 - Población Penitenciaria en Argentina




> http://diplodatos.famaf.unc.edu.ar/poblacion-penitenciaria-en-argentina-2002-a-2017/




## Práctico Aprendizaje Supervisado

En el práctico de Introducción al Aprendizaje Automático intentamos determinar la respuesta a la siguiente pregunta:

*¿Es posible predecir la duración de la condena que recibirá una persona en base a datos como los que se encuentran presentes en este dataset?*

Nuestra ingeniería de atributos fue sencilla: seleccionamos algunas variables que consideramos posiblemente correlacionadas a la variable objetivo y creamos modelos sencillos para tener una línea base para tener desde donde empezar. Intentamos modelar el problema tanto como regresión como clasificación, pero los resultados obtenidos no fueron satisfactorios. Raramente se superaron los resultados obtenidos por regresores o clasificadores dummy. 

En este práctico intentaremos mejorar los resultados iterando sobre la ingeniería de atributos, el modelado, y el análisis de la salida de los modelos.

Para simplificar sólo trabajaremos con la tarea de clasificación, usando la variable "duracion_condena_rango" como objetivo.


La siguiente es la definición de la variable a predecir:

**duracion_condena_rango**: si la situación legal del interno es condenado toma valores: 1) Hasta 3 años; Valor 2) De 3 a 6 años; Valor 3) De 6 a 9 años; 4) De 9 a 12 años; 5) De 12 a 15 años; 6) De 15 a 18 años; 7) Más de 18 años; 8) Sin datos. Si la situación legal del interno es procesado, inimputable o contraventor toma el valor 0. 

Fuente: https://github.com/datos-justicia-argentina/Sistema-Nacional-de-Estadisticas-sobre-Ejecucion-de-la-Pena-SNEEP/blob/master/Sistema-Nacional-de-Estadisticas-sobre-Ejecucion-de-la-Pena-SNEEP-metadata.md

### Resultados previos (completar)

Mejor accuracy obtenida en clasificación sobre el conjunto de Entrenamiento: 

0.0% 

Mejor accuracy obtenida en clasificación sobre el conjunto de Test: 

0.0%

### Carga del dataset

Cuando nos enfrentamos a resultados como los que obtuvimos en el primer práctico, una estrategia común es buscar mayor cantidad de datos. En esta oportunidad empezaraemos a trabajar con los datos completos desde 2002 hasta la fecha. 

In [1]:
import pandas as pd

In [2]:
data_raw = pd.read_csv('../datasets/sneep-unificado-2002-2017.csv')

/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (39,67,68,69,70,71,72,73,74,75,76,78,79,80,81,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Antes de poder utilizar el dataset realizaremos un conjunto de tareas:

- Corregir los problemas de columnas en los datos del censo 2017 que detectamos durante el análisis y curación de los datos.
- Procesar los campos de fechas para que se carguen con el tipo de datos correcto.
- Guardar sólo los registros que correspondan a personas condenadas.
- Remover duplicados. Esto es necesario porque la planilla contiene una concatenación de todos los censos, e internos que permanecen múltiples años en un penal van a aparecer reiteradas veces en el dataset.
- Imputar valores.
- Generar el atributo "anio_condenado", es una nueva variable que nos permitirá tener en cuenta la fecha aproximada en la cuál se condenó a la persona. 
- Generar el atributo "edad_al_ser_condenado", nueva variable que usaremos en lugar de la variable "edad", ya que ésta última no describe un atributo del interno al momento de ser dictada su sentencia si no el valor al momento del censo.
- Conservar sólo registros en los que la persona sea mayor de 15 años al momento de la condena, y cuya fecha de condena sea previa a la fecha del censo.
- Conservar registros de personas condenadas por un sólo delito. El objetivo es intentar modelar un problema más simple que el que planteamos anteriormente. Al desconocer el dominio judicial y no tener otros atributos que nos permitan entender cómo se modifica la sentencia cuando una persona comete múltiples delitos, primero intentaremos encontrar un modelo que pueda explicar el caso más sencillo.
- Corregir problemas con la variable objetivo. El valor 8 del atributo "duracion_condena_rango" significa "sin datos" y una buena parte de los registros con este valor no tienen datos en los campos "duracion_condena_anio" y "duracion_condena_meses". Sin embargo, sabemos que las cadenas perpetuas en Argentina son superiores a 18 años, así que todos los que tengan "tipo_condena" en valor 1, deben tener "duracion_condena_rango" en 7. El resto de los registros que no sean condenas perpetuas y no tengan valor definido serán eliminados.

In [3]:
def fix_2017(df):
    # acomodamos columnas en el csv de 2017
    data_2017 = df[df["anio_censo"] == 2017].copy()
    unique_index = pd.Index(list(col for col in data_2017.columns))
    index_tipo_condena = unique_index.get_loc('tipo_condena')
    for i in range(len(data_2017.columns)-1, index_tipo_condena, -1):
        data_2017.iloc[:,i] = data_2017.iloc[:,i-1]

    data = df[df["anio_censo"] != 2017]
    data.append(data_2017)
    return data
    
def parse_dates(df):
    df.loc[:,"fecha_detencion"] = pd.to_datetime(df["fecha_detencion"],errors='coerce')
    df.loc[:,"fecha_condenado"] = pd.to_datetime(df["fecha_condenado"],errors='coerce')
    return df

def get_condenados(df):
    data = df[df["situacion_legal_id"] == 1]
    return data

def remove_duplicates(df):
    filter_columns = ['delito1_id','provincia_id','establecimiento_id','genero_id','nacionalidad_id','jurisdiccion_id',
                      'fecha_detencion','fecha_condenado','capacitacion_laboral_al_ingresar_id']
    
    duplicated = df.duplicated(subset=filter_columns,keep='first')
    data = df[duplicated]
    return data

def impute_values(df):
    df['estado_civil_id'].fillna(0, inplace=True)
    df['nivel_instruccion_id'].fillna(0, inplace=True)
    df['ultima_situacion_laboral_id'].fillna(0, inplace=True)
    df['es_reincidente_id'].fillna(0, inplace=True)
    df['nivel_instruccion_id'].fillna(0, inplace=True)
    df['estado_civil_id'].fillna(0, inplace=True)
    df['edad'].fillna(df['edad'].mean(), inplace=True)
    df.replace({'edad': {0: df['edad'].mean()}},inplace=True) 
    
def calculate_anio_condenado(row):
    if row["fecha_detencion"] is pd.NaT:
        fecha = row["anio_censo"]
    else:
        fecha = row['fecha_detencion'].year + 1
    return fecha

def add_anio_condenado(df):
    df["anio_condenado"] = df.apply(lambda row: calculate_anio_condenado(row) if row['fecha_condenado'] is pd.NaT 
                                    else row['fecha_condenado'].year, axis=1)
    return df

def calculate_edad_al_ser_condenado(row):
    anios = row["anio_censo"] - row["anio_condenado"]
    edad_al_ser_condenando = row["edad"] - anios
    return edad_al_ser_condenando

def add_edad_al_ser_condenado(df):
    df["edad_al_ser_condenado"] = df.apply(lambda row: calculate_edad_al_ser_condenado(row), axis=1)
    return df

def remove_wrong_anio_o_edad(df):
    data = df[df["edad_al_ser_condenado"] > 15]
    data = data[data["anio_condenado"] <= data["anio_censo"]]
    return data    
    
def add_cantidad_delitos(df):
    delito_cols = ['delito{}_descripcion'.format(i) for i in range(1, 6)]
    df['delitos_cantidad'] = df[delito_cols].count(axis=1)
    return df

def filter_delitos(df):
    data = df[df["delitos_cantidad"] == 1]
    return data

def remove_rango_nodata(df):
    data = df
    data["duracion_condena_rango"] = data.apply(lambda row: 7 if row['tipo_condena'] == 1 else row['duracion_condena_rango'], axis=1)
    data = data[data["duracion_condena_rango"] != 8]
    return data

In [4]:
def preprocess_sneep(df):
    print("Cantidad de registros inicial:", len(df))
    data = fix_2017(df)
    data = parse_dates(data)
    data = get_condenados(data)
    print("Cantidad de registros de condenados:", len(data))    
    data = remove_duplicates(data)
    print("Cantidad de registros sin duplicados:", len(data))    
    impute_values(data)
    data = add_anio_condenado(data)
    data = add_edad_al_ser_condenado(data)
    data = add_cantidad_delitos(data)
    data = filter_delitos(data)
    print("Cantidad de registros con un solo delito:", len(data))
    data = remove_wrong_anio_o_edad(data)
    print("Cantidad de registros quitando anio de condena o edad equivocado:", len(data))
    data = remove_rango_nodata(data)
    print("Cantidad de registros quitando rango sin datos", len(data))
    return data

In [5]:
data = preprocess_sneep(data_raw)

Cantidad de registros inicial: 939727


/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Cantidad de registros de condenados: 385086
Cantidad de registros sin duplicados: 102935
Cantidad de registros con un solo delito: 82988
Cantidad de registros quitando anio de condena o edad equivocado: 77024
Cantidad de registros quitando rango sin datos 68755


In [6]:
# Guardamos el dataset curado para posteriores usos
data.to_csv('../datasets/sneep-unificado-2002-2017-CURADO.csv')

In [7]:
# Veamos cuantos registros hay en cada categoría
data["duracion_condena_rango"].value_counts()

2    20442
3    16874
4    10878
7     9141
5     5737
6     2886
1     2797
Name: duracion_condena_rango, dtype: int64

In [14]:
data_raw[['estado_civil_descripcion', 'estado_civil_id']].head()

,estado_civil_descripcion,estado_civil_id
0,Concubino,6.0
1,Concubino,6.0
2,Concubino,6.0
3,Concubino,6.0
4,Concubino,6.0


In [15]:
data_raw.estado_civil_descripcion.value_counts()

Soltero                  689837
Concubino                106518
Casado                    99560
Separado o divorciado     13987
Viudo                      9658
Separado de hecho          7211
Name: estado_civil_descripcion, dtype: int64

In [16]:
data_raw.estado_civil_id.value_counts()

1.0    689837
6.0    106518
2.0     99560
4.0     13987
0.0     12912
3.0      9658
5.0      7211
Name: estado_civil_id, dtype: int64

In [ ]:
estado_civil_map = {
    1: 'Soltero',
    2: 'Casado',
    3: 'Viudo',
    4: 'Separado',
    5: 
    6: 'Concubino'
}

In [17]:
data_raw.provincia_descripcion.value_counts()

Buenos Aires              472155
Córdoba                    96654
Mendoza                    49301
Santa Fe                   43743
Salta                      37526
Ciudad de Buenos Aires     29970
Chaco                      24028
Misiones                   20749
Río Negro                  18831
Entre Rios                 16986
Tucumán                    16153
San Juan                   14134
Neuquén                    12646
Chubut                     12432
Corrientes                 12430
Jujuy                      11722
La Pampa                    9971
Santiago del Estero         7500
Formosa                     7446
San Luis                    7404
Catamarca                   6860
Santa Cruz                  4823
La Rioja                    4088
Tierra del Fuego            2175
Name: provincia_descripcion, dtype: int64

In [18]:
data_raw.provincia_id.value_counts()

1     472155
3      96654
12     49301
20     43743
16     37526
24     29970
5      24028
13     20749
15     18831
7      16986
23     16153
17     14134
14     12646
6      12432
4      12430
9      11722
10      9971
21      7500
8       7446
18      7404
2       6860
19      4823
11      4088
22      2175
Name: provincia_id, dtype: int64

In [33]:
import numpy as np
from collections import Counter
prov_id   = data_raw.provincia_id.value_counts().index.values
prov_desc = data_raw.provincia_descripcion.value_counts().index.values 
dict(zip(prov_id, prov_desc))

{1: 'Buenos Aires',
 3: 'Córdoba',
 12: 'Mendoza',
 20: 'Santa Fe',
 16: 'Salta',
 24: 'Ciudad de Buenos Aires',
 5: 'Chaco',
 13: 'Misiones',
 15: 'Río Negro',
 7: 'Entre Rios',
 23: 'Tucumán',
 17: 'San Juan',
 14: 'Neuquén',
 6: 'Chubut',
 4: 'Corrientes',
 9: 'Jujuy',
 10: 'La Pampa',
 21: 'Santiago del Estero',
 8: 'Formosa',
 18: 'San Luis',
 2: 'Catamarca',
 19: 'Santa Cruz',
 11: 'La Rioja',
 22: 'Tierra del Fuego'}